<a href="https://colab.research.google.com/github/ejrtks1020/Computer_Vision/blob/main/_%EB%B3%80%EC%9D%B4%ED%98%95_%EC%98%A4%ED%86%A0%EC%9D%B8%EC%BD%94%EB%8D%94_(Variational_AutoEncoders%2C_VAE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이미지 생성 모델

- 이미지 잠재 공간에서 샘플링하여 완전히 새로운 이미지나 기존 이미지를 변형시키는 방식의 주요 기법
  - 변이형 오토인코더(Variational AutoEncoders, VAE)
  
  - 적대적 생성 네트워크(Generative Adversarial Networks, GAN)

- 잠재 공간의 한 포인트를 입력으로 받아 이미지(픽셀의 그리드)를 출력하는 모듈을 VAE에서는 디코더(decoder)라고 함

## 잠재 공간(Latent Space)

<img src="https://images.deepai.org/converted-papers/2007.08383/x1.png">

<sub>[이미지 출처] https://deepai.org/publication/deep-learning-in-protein-structural-modeling-and-design</sub>

# 변이형 오토인코더(Variational AutoEncoders, VAE)

- 입력 이미지를 잠재 공간의 고정된 코딩으로 압축하는 대신  
  이미지를 어떤 통계 분포의 파라미터로 변환 
  
- 평균과 분산 파라미터를 사용하여 이 분포에서 무작위로 하나의 샘플을 추출

- 해당 샘플을 디코딩하여 원본 입력으로 복원

- 위 과정은 안정성을 향상, 잠재 공간 어디에서든 의미있는 표현을 인코딩하게 함

  <img src="https://image.slidesharecdn.com/variationalautoencoder-170601084514/95/variational-autoencoder-21-638.jpg?cb=1496306885">

  <sub>[이미지 출력] https://www.slideshare.net/ssuser06e0c5/variational-autoencoder-76552518</sub>

  <br>

- VAE의 훈련

  - 디코딩된 샘플이 원본 입력과 동일하도록 만드는 재구성 손실(reconstruction loss)

  - 잠재 공간을 잘 형성하고 훈련 데이터에 과적합을 줄이는 규제 손실(regularization loss)

- VAE 구현의 pseudocode

      z_mean, z_log_var = encoder(input_img)
      
      z = z_mean + exp(0.5 * z_log_var) * epsilon

      recontructed_img = decoder(z)

      model = Model(input_img, reconstructed_img)

      재구성 손실과 규제 손실을 사용하여 훈련
  
  

# Convolutional VAE

- MNIST Dataset

- 코드 참조 : https://www.tensorflow.org/tutorials/generative/cvae?hl=ko

## 모듈 설치

In [ ]:
!pip install tensorflow-probability
!pip install imageio
!pip install git+https://github.com/tensorflow/docs

  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-7f2ijk6_
  Running command git clone -q https://github.com/tensorflow/docs /tmp/pip-req-build-7f2ijk6_
  Created wheel for tensorflow-docs: filename=tensorflow_docs-0.0.0.dev0-py3-none-any.whl size=157855 sha256=8a8a92f3f2b0df80f0663613b9b5c5ac6bc4052256615897e9c91bfe5f4ea0e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-7ttr8bo7/wheels/cc/c4/d8/5341e93b6376c5c929c49469fce21155eb69cef1a4da4ce32c
Successfully built tensorflow-docs


In [ ]:
from IPython import display

import glob
import imageio
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

import numpy as np
import PIL
import time

import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.data import Dataset
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape, Conv2DTranspose, Conv2D, InputLayer
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

## 데이터 로드

In [ ]:
(train_images , _) , (test_images , _ ) = mnist.load_data()

In [ ]:
print(train_images.shape)
print(test_images.shape)

(60000, 28, 28)
(10000, 28, 28)


## 데이터 전처리

In [ ]:
def preprocess_images(images):
  images = images.reshape((images.shape[0], 28, 28, 1)) / 255.
  return np.where(images > .5, 1.0, 0.0).astype('float32')

In [ ]:
train_images = preprocess_images(train_images)
test_images = preprocess_images(test_images)

In [ ]:
print(train_images.shape)
print(test_images.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [ ]:
train_size = 60000
batch_size = 32
test_size = 10000


## tf.data 를 사용하여 데이터 일괄 처리 및 섞기

In [ ]:
train_dataset = (Dataset.from_tensor_slices(train_images).shuffle(train_size).batch(batch_size))
test_dataset = (Dataset.from_tensor_slices(test_images).shuffle(test_size).batch(batch_size))

## 인코더, 디코더 모델 구성

- VAE 예제에서는 인코더 및 디코더 네트워크에 두 개의 작은 ConvNet을 사용

- 자세한 네트워크 구성 설명은 링크 참조
 
  - https://www.tensorflow.org/tutorials/generative/cvae?hl=ko

In [ ]:
class CVAE(tf.keras.Model):
  def __init__(self, latent_dim):
    super(CVAE , self).__init__()

    self.latent_dim = latent_dim
    self.encoder = Sequential([InputLayer(input_shape = (28,28,1)),
                               Conv2D(filters = 32, kernel_size = 3,
                                      strides = (2, 2), activation = 'relu'),
                               Conv2D(filters = 64, kernel_size = 3,
                                      strides = (2, 2), activation = 'relu'),
                               Flatten(),
                               Dense(latent_dim + latent_dim)])
    
    self.decoder = Sequential([InputLayer(input_shape = (latent_dim,)),
                               Dense(units = 7*7*32 , activation = 'relu'),
                               Reshape(target_shape = (7, 7, 32)),
                               Conv2DTranspose(filters=64, kernel_size = 3,
                                               strides = 2, padding = 'same',
                                               activation = 'relu'),
                               Conv2DTranspose(filters = 64, kernel_size = 3,
                                               strides = 2, padding = 'same', 
                                               activation = 'relu'),
                               Conv2DTranspose(filters = 1, kernel_size = 3,
                                               strides =1 , padding = 'same')])
    
    def sample(self ,eps = None):
      if eps is None:
        eps = tf.random.normal(shape = (100 ,self.latent_dim))
      return self.decode(eps, apply_sigmoid = True)

    def encode(self, x):
      mean, logvar = tf.split(self.encoder(x), num_or_size_splits = 2, axis = 1)

    
    def reparameterize(self, mean, logvar):
      eps = tf.random.normal(shape = mean.shape)
      return eps * tf.exp(logvar * .5) + mean

    def decode(self, z, apply_sigmoid = False):
      logits = self.decoder(z)

      if apply_sigmoid:
        probs = tf.sigmoid(logits)
        return probs
      
      return probs

    

## 손실 함수 및 최적화 프로그램 정의

- VAE는 한계 로그 우도에서 증거 하한 (ELBO)을 최대화하여 학습

$$\log p(x) \ge \text{ELBO} = \mathbb{E}_{q(z|x)}\left[\log \frac{p(x, z)}{q(z|x)}\right].$$

- 실제로 우리는이 기대치의 단일 샘플 Monte Carlo 추정치를 최적화

$$\log p(x| z) + \log p(z) - \log q(z|x),$$




In [ ]:
optimizer = Adam(1e-4)

In [ ]:
def log_normal_pdf(sample, mean, logvar, raxis = 1):
  log2pi = tf.math.log(2. * np.pi)
  return tf.reduce_sum( -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi), axis = raxis)

def compute_loss(model, x):
  mean, logvar = model.encode(x)
  z = model.reparameterize(mean,logvar)
  x_logit = model.decode(z)
  cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits = logit, labels = x)
  logpx_z = tf.reduce_sum(cross_ent, axis = [1,2,3])
  logpz = log_normal_pdf(z, 0., 0.)
  logqz_x = log_normal_pdf(z, mean, logvar)

  return -tf.reduce_mean(logpx_z +logpz - logqz_x)

@tf.function
def train_step(model, x, optimizer):
  with tf.GradientTape() as tape:
    loss = compute_loss(model, x)
  
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

## 모델 학습 및 이미지 생성

## 학습 시각화


- 마지막 훈련 스텝에서 생성 된 이미지 표

- 저장된 모든 이미지의 애니메이션 GIF 표시

- 잠재 공간에서 2D 다양한 숫자 표시